In [ ]:
__version__ = '1.0.0-alpha.1'
__author__ = 'Martino Pulici'

In [ ]:
import csv

In [ ]:
import pandas as pd
import pulp

In [ ]:
from branch_and_bound import branch_and_bound

In [ ]:
prob_day_1 = pulp.LpProblem("Day_1", pulp.LpMinimize)
prob_day_2 = pulp.LpProblem("Day_2", pulp.LpMinimize)
prob_day_3 = pulp.LpProblem("Day_3", pulp.LpMinimize)
prob_day_4 = pulp.LpProblem("Day_4", pulp.LpMinimize)
prob_day_5 = pulp.LpProblem("Day_5", pulp.LpMinimize)
probs = [prob_day_1, prob_day_2, prob_day_3, prob_day_4, prob_day_5]

In [ ]:
df = pd.read_csv("food.csv")
with open("dish.csv") as f:
    reader = csv.reader(f)
    dish_list = list(reader)
with open("unlimited.txt") as f:
    reader = csv.reader(f)
    unlimited_dishes = list(reader)[0]

In [ ]:
def unlimited_dish(string):
    for food in unlimited_dishes:
        if food in string:
            return True
    return False

In [ ]:
nutrient_limits = {'Energy': 2000,
                   'Fat': 70,
                   'Saturates': 20,
                   'Carbohydrates': 260,
                   'Sugars': 90,
                   'Protein': 50
                   }

In [ ]:
class dish:
    def __init__(self, name):
        self.name = name
        self.cost = 0
        self.nutrients = {nut: 0 for nut in nutrient_limits.keys()}

In [ ]:
labels = list(df['Food'])
costs = dict(zip(labels, df['Cost']))

In [ ]:
nutrients = {}
for i in range(len(labels)):
    nutrients[labels[i]] = {}
    for nut in nutrient_limits.keys():
        nutrients[labels[i]][nut] = df[nut][i] * 10

In [ ]:
dish_labels = []
dishes = []
for i in range(len(dish_list)):
    d = dish_list[i]
    dishes.append(dish(d[0]))
    dish_labels.append(d[0])
    for j in range(1, len(d), 2):
        ingredient = d[j]
        quantity = float(d[j + 1]) / 1000
        dishes[i].cost += costs[ingredient] * quantity
        for nut in dishes[i].nutrients.keys():
            dishes[i].nutrients[nut] += nutrients[ingredient][nut] * quantity

In [ ]:
food_day_1 = pulp.LpVariable.dicts("Food_Day_1", dish_labels, 0)
food_day_2 = pulp.LpVariable.dicts("Food_Day_2", dish_labels, 0)
food_day_3 = pulp.LpVariable.dicts("Food_Day_3", dish_labels, 0)
food_day_4 = pulp.LpVariable.dicts("Food_Day_4", dish_labels, 0)
food_day_5 = pulp.LpVariable.dicts("Food_Day_5", dish_labels, 0)
foods = [food_day_1, food_day_2, food_day_3, food_day_4, food_day_5]

In [ ]:
for i in range(len(probs)):
    probs[i] += pulp.lpSum([dish.cost * foods[i][dish.name] for dish in dishes])
    for nut in nutrient_limits.keys():
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name] for dish in dishes]) >= nutrient_limits[nut] * 0.67
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name] for dish in dishes]) <= nutrient_limits[nut] * 1.5
    for j in range(len(probs[i].variables())):
        probs[i] += probs[i].variables()[j] <= 3

In [ ]:
new = []

for i in range(0, len(probs)):
    for j in range(0, i):
        for k in range(len(new[j].variables())):
            if new[j].variables()[k].varValue and not unlimited_dish(str(new[j].variables()[k])):
                probs[i] += probs[i].variables()[k] == 0
            if (j == i - 1 or j == i - 2) and new[j].variables()[k].varValue and unlimited_dish(str(new[j].variables()[k])):
                probs[i] += probs[i].variables()[k] == 0
    new.append(branch_and_bound(probs[i]))
    new[-1].solve()

In [ ]:
print("Cost = " + str(pulp.value(sum([n.objective for n in new]))) + " €")

In [ ]:
for day in ["_Day_1_", "_Day_2_", "_Day_3_", "_Day_4_", "_Day_5_"]:
    print()
    print(day[1:6].upper())
    for n in new:
        for v in n.variables():
            if v.varValue and day in v.name:
                print(v.name[11:] + " = {0:.0f}".format(v.varValue))